In [1]:
import pretty_midi_fluidsynth as pmf
import librosa
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio

In [2]:
sr = 44100

hop_len = 512
freq_w = 3
octave = 8

fs = 100

In [3]:
df_dict = {'inputs':[],
           'outputs':[]}

In [4]:
for _ in range(100):
    instr_list = []
    
    n = np.random.randint(1,10)
    for _ in range(n):
        i = np.random.randint(128)
        instr = pmf.Instrument(program=i)
        instr.name = pmf.program_to_instrument_name(i)
        
        note_n = np.random.randint(2,10)
        for _ in range(note_n):
            velocity = np.random.randint(50,120)
            pitch = np.random.randint(24,120)
            start_time = np.random.randint(1,10)
            end_time = start_time + np.random.randint(1,4)
            note = pmf.Note(velocity=velocity,
                           pitch=pitch,
                           start=start_time,
                           end=end_time,)
            instr.notes.append(note)
        instr_list.append(instr)
    
    pm = pmf.PrettyMIDI()
    for instr in instr_list:
        pm.instruments.append(instr)

    y = pm.get_piano_roll(fs)

    wave_format = pm.fluidsynth(sr)

    x = librosa.cqt(
        wave_format, sr=sr,
        hop_length= hop_len,
        n_bins= 12*freq_w*octave,
        bins_per_octave= 12*freq_w
    )

    df_dict['inputs'].append(x)
    df_dict['outputs'].append(y)

In [5]:
print(f'{len(df_dict["inputs"])=}')
print(f'{len(df_dict["outputs"])=}')

len(df_dict["inputs"])=100
len(df_dict["outputs"])=100


In [6]:
print(f'{df_dict["inputs"][0].shape=}')
print(f'{df_dict["outputs"][0].shape=}')

df_dict["inputs"][0].shape=(288, 1120)
df_dict["outputs"][0].shape=(128, 1200)
